In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from ipywidgets import Layout, HTML
from IPython.core.interactiveshell import InteractiveShell
from ipyevents import Event
import random
import threading

In [2]:
# Setup global configurations
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 5)

In [3]:
# Load empty board
height = 10
width = 20
df = pd.DataFrame(np.array(list(" " * height * width)).reshape(height,width))

# Initialize disk
disk = {"x":height//2-1, "y":width//3, "img":"💾"}
df.iloc[(disk["x"], disk["y"])] = disk["img"]

In [4]:
# Initialize board
output = widgets.Output()
board = widgets.HBox([output], layout=Layout(height='500px'))
def refreshBoard(source=None):
    output.clear_output(wait=True)
    with output:
        display(df)

In [5]:
speed = 3.0
def dropDisk():
    global disk
    nextX = disk["x"] + 1
    if nextX < height:
        df.iloc[(disk["x"], disk["y"])] = " "
        disk["x"] = nextX
        df.iloc[(disk["x"], disk["y"])] = disk["img"]
    refreshBoard()
    timer = threading.Timer(speed, dropDisk)
    timer.start()

In [6]:
# Start game 
def startGame():
    display(board)
    refreshBoard()
    dropDisk()

In [7]:
startGame()

In [8]:
disk

{'x': 5, 'y': 6, 'img': '💾'}

In [9]:
refreshBoard()